# Create all Features for Items

packages

In [1]:
import pandas as pd
import numpy as np
import gensim
import feather
import nltk
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import sys
#!{sys.executable} -m pip install bs4
from bs4 import BeautifulSoup

load in data - all articles that appeared in the target list (train and test)

In [4]:
items = pd.read_pickle("data/processed_data/articles_list.pkl")

In [5]:
items_df = pd.DataFrame(items)
items_df.columns = ['anbieter_artikelnummer']

In [33]:
len(items_df)

750206

In [34]:
items_df.head()

,anbieter_artikelnummer
0,0000400435550
1,000040053900017-002
2,0000403433309
3,0000403434036
4,00004034LB-03


## What do I need?

Features i want:
--------------------------

Categorical:
    - supplier id ✔
    - supplier marketplace ✔
    - product category ✔
    - buyer group categories - Leave out, to many NaN
Continouus:
    - price ✔
    - min PU ✔
---------------------------------

Embeddings:
    - image - Leave out, maybe later
    - text description ✔
    
Calculated with Log Table: 
    - days online 
    - purchases last 30days 

## General Item Information

### 1. SupplierID, Price, Days online, Min PU

### Data

In [11]:
items_info_all = feather.read_dataframe("data/raw_data/artikelstammdaten.feather")

In [35]:
len(items_info_all)

16669411

In [36]:
items_info_all.head()

,artikelID,artikelnummer,artikelname,anbieterID,anbieter_artikelnummer,pfi,marktplatz,stueck_pro_ve,stueckpreis,waehrung,rabatt,erstanlageDatum,letztesAktualisierungsDatum,status,ean,isCrossorder,isVariant,statistischeWarennummer,gewicht
0,4014156,059-04126,Cowboy-Zubehör,8,00000008059-04126,265.5834,DE,1.0,9.80,PLN,0.0,2012-10-13 21:15:45+00:00,2014-01-14 15:01:41+00:00,Account gesperrt,None,0,0,None,0.0
1,4014170,059-04133,AUTO RALLY ON THE RADIO,8,00000008059-04133,263.6376,DE,1.0,18.85,PLN,0.0,2012-10-13 21:15:50+00:00,2014-01-14 15:22:30+00:00,Account gesperrt,None,0,0,None,0.0
2,4014184,059-04157,Bahnhof mit der Batterie,8,00000008059-04157,265.5834,DE,1.0,11.82,PLN,0.0,2012-10-13 21:15:55+00:00,2014-01-14 15:07:41+00:00,Account gesperrt,None,0,0,None,0.0
3,4014198,059-04164,Zug nach BATTERIE,8,00000008059-04164,500.0000,DE,1.0,9.92,PLN,0.0,2012-10-13 21:16:00+00:00,2014-01-14 15:02:09+00:00,Account gesperrt,None,0,0,None,0.0
4,4014212,059-04256,POLIZEI AUTO CARTON,8,00000008059-04256,500.0000,DE,1.0,6.30,PLN,0.0,2012-10-13 21:16:06+00:00,2014-01-14 14:49:16+00:00,Account gesperrt,None,0,0,None,0.0


In [10]:
# with filter - didnt work that well... 
#items_info = items_info_all[items_info_all.marktplatz == "EU"]

In [11]:
#items_info_grouped = items_info_all.groupby(['anbieter_artikelnummer'])['anbieterID', 'stueckpreis', 'waehrung', 'erstanlageDatum', 'stueck_pro_ve', 'artikelID'].min().reset_index()

In [12]:
len(items_info)

2050735

In [13]:
items_info_filtered = items_info_all.drop_duplicates(subset=['anbieter_artikelnummer'])

### Join with items

In [37]:
items_plus = pd.merge(items_df, items_info_filtered, how = "left", on = ['anbieter_artikelnummer'])

In [39]:
len(items_plus)

750206

In [64]:
sum(pd.isna(items_plus.artikelID)) # articles that where not found in data... 

1174

In [41]:
items_plus.head()

,anbieter_artikelnummer,artikelID,artikelnummer,artikelname,anbieterID,pfi,marktplatz,stueck_pro_ve,stueckpreis,waehrung,rabatt,erstanlageDatum,letztesAktualisierungsDatum,status,ean,isCrossorder,isVariant,statistischeWarennummer,gewicht
0,0000400435550,11608786.0,35550,Koffergurt mit Klettverschluss bunt,4004.0,710.103900,DE,12.0,2.10,EUR,0.0,2016-03-04 14:39:47+00:00,2018-08-17 08:33:01+00:00,Account freigegeben,,0.0,0.0,None,NaN
1,000040053900017-002,17618200.0,3900017-002,"Leonardo Verrelli Echt Leder Tasche, Farbe: 2",4005.0,601.972400,DE,1.0,59.95,EUR,0.0,2018-07-09 12:51:10+00:00,2019-09-13 14:47:07+00:00,Account freigegeben,None,0.0,0.0,None,NaN
2,0000403433309,18402615.0,33309,Teelichthalter Buddha,4034.0,600.923100,DE,12.0,2.79,EUR,0.0,2018-11-08 13:04:42+00:00,2018-11-08 13:04:42+00:00,Account freigegeben,None,0.0,0.0,None,NaN
3,0000403434036,13334081.0,34036,Weihnachts LED Lichterkette aus Kupferdraht 3m...,4034.0,610.470391,DE,20.0,2.10,EUR,30.0,2016-11-29 15:31:47+00:00,2019-08-14 11:04:57+00:00,Account gesperrt,,0.0,0.0,None,NaN
4,00004034LB-03,16079725.0,LB-03,LED Ballon mit Lichterkette,4034.0,747.432300,DE,10.0,1.89,EUR,0.0,2017-11-15 14:27:01+00:00,2019-08-28 10:42:34+00:00,Account freigegeben,,0.0,1.0,None,NaN


In [43]:
items_plus_final = items_plus[['anbieter_artikelnummer', 'anbieterID', 'stueckpreis', 'waehrung', 'erstanlageDatum', 'stueck_pro_ve', 'artikelID']]

In [44]:
items_plus_final.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0


In [45]:
items_plus_final.to_pickle('data/processed_data/articles_info.pkl')

## 2. Supplier MKT

In [50]:
supplier = pd.read_csv('data/raw_data/anbieterstammdaten.csv', sep = ";", encoding = 'ISO-8859-15')

In [51]:
supplier.head()

,Unnamed: 0,anbieterID,anbietermarktplatz,firma,anbieterLand,plz,anbieterStatus,distri,erstanlageDatum,angebotsfunktion,qualitaetsindex,salestools
0,1,4004,DE,PM Euro-Trading GmbH,Deutschland,36100,aktiv,Nein,-001-11-30 00:53:28,Ja,1,Ja
1,2,4005,DE,Shaghafi GmbH,Deutschland,48431,aktiv,Nein,-001-11-30 00:53:28,Ja,"0,8",Ja
2,3,4034,DE,"China Trading, C&T Handels GmbH",Deutschland,50858,aktiv,Nein,2002-11-20 01:44:00,Ja,1,Ja
3,4,4045,DE,Haroma-Oil GmbH,Deutschland,52531,aktiv,Nein,2002-11-22 15:17:00,Nein,1,Nein
4,5,4068,DE,MB-Müller Jürgen Müller e.K.,Deutschland,69214,aktiv,Nein,2002-12-10 14:14:00,Ja,1,Ja


In [53]:
supplier_imp = supplier[['anbieterID', 'anbietermarktplatz']]

In [54]:
items_supplier = pd.merge(items_plus_final, supplier_imp, how = "left", on = "anbieterID")

In [55]:
len(items_supplier)

750206

In [56]:
items_supplier.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE


In [57]:
items_supplier.to_pickle('data/processed_data/articles_info.pkl')

## 3. Product Categories

In [58]:
article_groups = pd.read_csv('data/raw_data/artikelwarengruppe.csv', sep = ";", encoding = 'ISO-8859-15')

In [59]:
article_groups.head()

,Unnamed: 0,artikelID,warengruppe
0,1,79352,Bälle & Schläger_5185
1,2,116985,Gartendeko & Beleuchtung_5064
2,3,149863,Dekoration_5103
3,4,207328,Ordnung & Aufbewahrung_5140
4,5,210480,Fahrräder & Zubehör_5187


In [60]:
len(article_groups)

6890691

In [61]:
items_cat = pd.merge(items_supplier, article_groups, how = "left", on = "artikelID")

In [62]:
len(items_cat)

750206

In [67]:
sum(pd.isna(items_cat.warengruppe)) # articles that where not found in warengruppen... 

34865

In [68]:
items_cat.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,Unnamed: 0,warengruppe
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE,854601.0,Reiseartikel_5199
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE,4123796.0,sonstige Taschen_5202
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE,4861663.0,Kerzen & Kerzenhalter_5107
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE,1387339.0,Lichterketten_5114
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE,2914086.0,Lichterketten_5114


In [71]:
items_cat.drop('Unnamed: 0', axis= 1, inplace = True)

In [72]:
items_cat.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,warengruppe
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE,Reiseartikel_5199
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE,sonstige Taschen_5202
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE,Kerzen & Kerzenhalter_5107
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE,Lichterketten_5114
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE,Lichterketten_5114


In [73]:
items_cat.to_pickle('data/processed_data/articles_info.pkl')

## 4. Buyer Groups

In [74]:
buyer_groups = pd.read_csv('data/raw_data/artikelbuyersgroup.csv', sep = ";", encoding = 'ISO-8859-15')

In [76]:
buyer_groups.drop('Unnamed: 0', axis = 1, inplace = True)

In [88]:
buyer_groups.sample(10)

,artikelID,buyersworld,untergruppe,groupID
633824,18445889,Kiosk- und Thekenware,Schlüsselanhänger,799
1361381,19081074,Home & Living,Bettwäsche,169
2461222,16536101,Die große Welt der Lizenzartikel,Star Wars,516
1248645,16629473,Fashion & Accessoires,KAROLINA MROWINSKA,743
1975568,18797641,Spielwaren & Geschenkartikel,Glückwunschkarten,333
1312307,3799809,Discount-Fashion,"Schals, Tücher, Mützen, Handschuhe",131
387197,17681492,Discount-Fashion,Trenduhren,135
1200316,13245295,Die große Welt der Lizenzartikel,Disney,479
2083035,16895405,Drogerie & Haushaltswaren,Gläser,566
1466357,19807470,Saison- und Aktionsthemen,Geschenkverpackung & Karten,390


In [85]:
len(pd.unique(buyer_groups.groupID))

464

In [86]:
len(buyer_groups)

2511414

Every item can have multiple buyer groups ! 

In [87]:
len(pd.unique(buyer_groups.artikelID))

1724923

In [90]:
buyer_groups_two = buyer_groups.groupby(['artikelID']).groupID.agg([min,max]).reset_index()


In [92]:
buyer_groups_two.columns = ['artikelID', 'groupID_1', 'groupID_2']

In [93]:
buyer_groups_two.head()

,artikelID,groupID_1,groupID_2
0,79352,371,371
1,132494,254,254
2,211589,371,371
3,211592,371,371
4,211685,254,254


In [97]:
items_buyerGroups = pd.merge(items_cat, buyer_groups_two, how = "left", on = "artikelID")

In [98]:
len(items_buyerGroups)

750206

In [99]:
items_buyerGroups.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,warengruppe,groupID_1,groupID_2
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE,Reiseartikel_5199,NaN,NaN
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE,sonstige Taschen_5202,NaN,NaN
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE,Kerzen & Kerzenhalter_5107,190.0,878.0
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE,Lichterketten_5114,NaN,NaN
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE,Lichterketten_5114,385.0,405.0


In [100]:
items_buyerGroups.to_pickle('data/processed_data/articles_info.pkl')

## 5. Calculate Price

In [ ]:
# need to include exchange rate based on waehrung - multiple ifs - have three different currencies
#items_buyerGroups['preis'] = [items_buyerGroups.stueckpreis * 0.228469 if x == 'PLN' else 'NaN' for x in items_buyerGroups['waehrung']]

In [4]:
items_buyerGroups = pd.read_pickle("data/processed_data/articles_info.pkl")

In [5]:
items_buyerGroups.head()

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,warengruppe,groupID_1,groupID_2
0,0000400435550,4004.0,2.10,EUR,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE,Reiseartikel_5199,NaN,NaN
1,000040053900017-002,4005.0,59.95,EUR,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE,sonstige Taschen_5202,NaN,NaN
2,0000403433309,4034.0,2.79,EUR,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE,Kerzen & Kerzenhalter_5107,190.0,878.0
3,0000403434036,4034.0,2.10,EUR,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE,Lichterketten_5114,NaN,NaN
4,00004034LB-03,4034.0,1.89,EUR,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE,Lichterketten_5114,385.0,405.0


In [6]:
items_buyerGroups['exchangeRate'] = np.where(items_buyerGroups['waehrung'] == 'EUR', 1 , 0)

In [8]:
items_buyerGroups['exchangeRate'] = np.where(items_buyerGroups['waehrung'] == 'PLN', 0.228786 , items_buyerGroups['exchangeRate'] )

In [10]:
items_buyerGroups['exchangeRate'] = np.where(items_buyerGroups['waehrung'] == 'HUF', 0.00299247, items_buyerGroups['exchangeRate'] )

In [13]:
items_buyerGroups['preis_euro'] = items_buyerGroups['stueckpreis'] * items_buyerGroups['exchangeRate']

In [14]:
items_buyerGroups.sample(50)

,anbieter_artikelnummer,anbieterID,stueckpreis,waehrung,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,warengruppe,groupID_1,groupID_2,exchangeRate,preis_euro
289151,00485179TB09 digital .. /,485179.0,45.00,PLN,2017-06-12 12:38:39+00:00,1.0,14977224.0,PL,Koffer & Trolleys_5197,NaN,NaN,0.228786,10.295370
294690,0044709923327/BND-16042,447099.0,3.19,PLN,2017-09-28 09:32:56+00:00,1.0,15764473.0,PL,NaN,NaN,NaN,0.228786,0.729827
236993,0056442965652345561,564429.0,0.63,EUR,2018-08-08 16:06:51+00:00,12.0,17753757.0,PL,Socken & Strumpfhosen_5231,479.0,723.0,1.000000,0.630000
418792,0013700295318-36/91,137002.0,19.99,EUR,2016-12-23 11:39:43+00:00,2.0,13475130.0,DE,Blumentöpfe & Vasen_5102,NaN,NaN,1.000000,19.990000
473747,001930348113,193034.0,19.80,EUR,2015-01-14 08:03:29+00:00,2.0,9115432.0,DE,Küchenutensilien_5136,707.0,707.0,1.000000,19.800000
548563,00497049ABYTEX511-XL,497049.0,12.20,EUR,2018-10-17 13:30:19+00:00,1.0,18300848.0,DE,Shirts & Tops_5055,NaN,NaN,1.000000,12.200000
477760,001838268718658234556,183826.0,2.49,EUR,2019-04-30 10:19:27+00:00,6.0,19424089.0,NL,Dekoration_5103,500.0,500.0,1.000000,2.490000
503871,005644296565241010,564429.0,1.87,EUR,2018-03-03 21:01:56+00:00,12.0,16707504.0,PL,Tücher & Schals _5059,131.0,757.0,1.000000,1.870000
112915,00690856HDZ-10010,690856.0,0.38,EUR,2018-06-13 16:09:03+00:00,10.0,17507774.0,DE,Schuhzubehör_5205,NaN,NaN,1.000000,0.380000
138994,0044709914019/KWS-14126,447099.0,3.99,PLN,2018-01-23 09:39:32+00:00,1.0,16464220.0,PL,Dekoration_5103,NaN,NaN,0.228786,0.912856


In [15]:
items_price = items_buyerGroups.drop(['stueckpreis', "waehrung", "exchangeRate"], axis = 1)

In [20]:
len(items_price)

750206

In [21]:
items_price.to_pickle('data/processed_data/articles_info.pkl')

In [19]:
# Calculate days online is diff(erstanlageDatum, click_datum) but done in training log

# Final DF

In [3]:
items_info = pd.read_pickle ('data/processed_data/articles_info.pkl')

In [4]:
items_info.head()

,anbieter_artikelnummer,anbieterID,erstanlageDatum,stueck_pro_ve,artikelID,anbietermarktplatz,warengruppe,groupID_1,groupID_2,preis_euro,text_clean,text_vec
0,0000400435550,4004.0,2016-03-04 14:39:47+00:00,12.0,11608786.0,DE,Reiseartikel_5199,NaN,NaN,2.10,Luggage strap with velcro colorful A tear-resi...,"[0.7984229, 0.09205151, -0.30848747, 0.669923,..."
1,000040053900017-002,4005.0,2018-07-09 12:51:10+00:00,1.0,17618200.0,DE,sonstige Taschen_5202,NaN,NaN,59.95,"Leonardo Verrelli Genuine Leather Bag, Color: ...","[0.19540575, -0.07194629, -0.34633487, -0.2459..."
2,0000403433309,4034.0,2018-11-08 13:04:42+00:00,12.0,18402615.0,DE,Kerzen & Kerzenhalter_5107,190.0,878.0,2.79,Tealight holder Buddha Tealight holder Buddha ...,"[0.5111147, 0.018874569, 0.22680217, 0.7675127..."
3,0000403434036,4034.0,2016-11-29 15:31:47+00:00,20.0,13334081.0,DE,Lichterketten_5114,NaN,NaN,2.10,Christmas LED fairy lights made of copper wire...,"[0.34992826, -0.059115075, -0.25212124, -0.110..."
4,00004034LB-03,4034.0,2017-11-15 14:27:01+00:00,10.0,16079725.0,DE,Lichterketten_5114,385.0,405.0,1.89,LED balloon with fairy lights Transparent ball...,"[0.13192588, -0.19249268, -0.3639368, -0.47083..."


In [5]:
len(items_info)

750206

Filter items that lack basic features: 
- supplier
- supplier marketplace

In [9]:
items_info = items_info[(pd.isna(items_info.anbieterID) == False)]

In [10]:
items_info = items_info[(pd.isna(items_info.anbietermarktplatz) == False)]

In [11]:
len(items_info)

749032

Drop features that are omitted from model

In [12]:
items_info.drop(['artikelID', 'groupID_1', 'groupID_2', "text_clean"], axis = 1, inplace = True)

In [13]:
items_info.head()

,anbieter_artikelnummer,anbieterID,erstanlageDatum,stueck_pro_ve,anbietermarktplatz,warengruppe,preis_euro,text_vec
0,0000400435550,4004.0,2016-03-04 14:39:47+00:00,12.0,DE,Reiseartikel_5199,2.10,"[0.7984229, 0.09205151, -0.30848747, 0.669923,..."
1,000040053900017-002,4005.0,2018-07-09 12:51:10+00:00,1.0,DE,sonstige Taschen_5202,59.95,"[0.19540575, -0.07194629, -0.34633487, -0.2459..."
2,0000403433309,4034.0,2018-11-08 13:04:42+00:00,12.0,DE,Kerzen & Kerzenhalter_5107,2.79,"[0.5111147, 0.018874569, 0.22680217, 0.7675127..."
3,0000403434036,4034.0,2016-11-29 15:31:47+00:00,20.0,DE,Lichterketten_5114,2.10,"[0.34992826, -0.059115075, -0.25212124, -0.110..."
4,00004034LB-03,4034.0,2017-11-15 14:27:01+00:00,10.0,DE,Lichterketten_5114,1.89,"[0.13192588, -0.19249268, -0.3639368, -0.47083..."


In [14]:
items_info.to_pickle('data/training_data/articles_info_final.pkl')

In [15]:
items_info.to_pickle('data/testing_data/articles_info_final.pkl')

## 6. Text Embedding

Use Title (bezeichnung) and shortDescription (beschreibung) of each article

In [2]:
items_df = pd.read_pickle ('data/training_data/articles_info_final.pkl')

In [3]:
descriptions = feather.read_dataframe("data/raw_data/artikeltexte.feather")

In [4]:
len(items_df)

749032

In [5]:
len(descriptions)

1752124

In [6]:
descriptions.head()

,artikelID,bezeichnung,beschreibung,artikelLangText,markenstichwort,artikelnummer,faname1,ausRufname,shopGroups,marktplatz,firmaID
0,5766,Thermolockenwickler - Umweltschonend ohne Strom,Schnelle und voluminöse Frisur nach neuster Mo...,Die Thermo-Lockenwickler ohne Klammer kurz im ...,,054,GoForm GmbH,GoForm,,DE,4221
1,6208,"Mini-Glitter-Ventilator,***Mini-Glitter-Ventil...","Mini-Glitter-Ventilator, ...die Disco-Variante",mit Glitter in transparentem Kunststoff <br>- ...,,40018,Hegenberg Trendcenter,Hegenberg,,DE,3997
2,6210,"Mini-Laser-Ventilator,***Mini-Laser-Ventilator...","Mini-Laser-Ventilator, mit Farbspiel",mit Mini-LEDs auf dem Propellerkopf<br>- erzeu...,,40021,Hegenberg Trendcenter,Hegenberg,,DE,3997
3,6279,"Mini-Chrom-Ventilator,***Mini-Chrom-Ventilator...",Die kühle Brise....,kleiner Ventilator<br>- in Chrom-Optik<br>- mi...,,40017,Hegenberg Trendcenter,Hegenberg,,DE,3997
4,6349,Koffergurte mit Namensschild,Knallerpreis: Koffergurte einfarbig und zweifa...,,,70040-00,Krüger-Import GmbH,Krüger Import,,DE,4375


filter for EU

In [7]:
descriptions_en = descriptions[descriptions.marktplatz == "EU"]

In [8]:
descriptions_en.head()

,artikelID,bezeichnung,beschreibung,artikelLangText,markenstichwort,artikelnummer,faname1,ausRufname,shopGroups,marktplatz,firmaID
890920,12730316,JACKET MAN ULTRA LIGHT BY MTX,Model wears a S - waisted cut - lightweight ma...,COLOR: RED SIZE: M | L | XL | XXL DISTRIBUTION...,MTX,D178 RED,SARL Falida,MTX Mentex,,EU,513388
1705803,19782931,100539 Silk-ka Magnolia branch pink with snow ...,Magnolia branch of the Silk-ka brand. <br /><b...,,,8718648005395,Handelsonderneming Chiel Meekes B.V.,Chiel Meekes,"Artificial flowers,Decoration",EU,183826
1466257,17910262,3 glass angel h = 7cm in box (set price),"3 glass angels, height: 7cm in box (set price)...",,,5515,IKO by zentrada.distribution,IKO,"Glass + Glass vases,Glass,Houseware,assorted b...",EU,765115
910877,13015506,Hand puppet king,"Material: fabric, Gender: female, Packing: Tra...",Curtain on! The puppet theater can start and i...,small foot,10236,Legler OHG small foot company,Legler_OHG,"Hand puppets and aperitifs,Dolls and plush items",EU,61824
1392979,17354707,"Cotton-Flickstoff iron-on, 11 x 25cm, beige","Cotton-iron-on patch, 11 x 25cm, beige",Outer carton: 10 pieces,,HDZ-07052,Heinrich Duscha OHG,H. Duscha oHG,"Ribbons, Cord, Patches,Ribbons, Cord, Patches",EU,690856


create AnbieterArtikelnummer for merge with items_df

In [9]:
descriptions_en = descriptions_en.astype({'firmaID' : 'str'})

In [10]:
descriptions_en['firmaID'] = descriptions_en.firmaID.str.pad(8,fillchar='0') 


In [11]:
descriptions_en['anbieter_artikelnummer'] = descriptions_en.firmaID + descriptions_en.artikelnummer

In [12]:
descriptions_en.head()

,artikelID,bezeichnung,beschreibung,artikelLangText,markenstichwort,artikelnummer,faname1,ausRufname,shopGroups,marktplatz,firmaID,anbieter_artikelnummer
890920,12730316,JACKET MAN ULTRA LIGHT BY MTX,Model wears a S - waisted cut - lightweight ma...,COLOR: RED SIZE: M | L | XL | XXL DISTRIBUTION...,MTX,D178 RED,SARL Falida,MTX Mentex,,EU,00513388,00513388D178 RED
1705803,19782931,100539 Silk-ka Magnolia branch pink with snow ...,Magnolia branch of the Silk-ka brand. <br /><b...,,,8718648005395,Handelsonderneming Chiel Meekes B.V.,Chiel Meekes,"Artificial flowers,Decoration",EU,00183826,001838268718648005395
1466257,17910262,3 glass angel h = 7cm in box (set price),"3 glass angels, height: 7cm in box (set price)...",,,5515,IKO by zentrada.distribution,IKO,"Glass + Glass vases,Glass,Houseware,assorted b...",EU,00765115,007651155515
910877,13015506,Hand puppet king,"Material: fabric, Gender: female, Packing: Tra...",Curtain on! The puppet theater can start and i...,small foot,10236,Legler OHG small foot company,Legler_OHG,"Hand puppets and aperitifs,Dolls and plush items",EU,00061824,0006182410236
1392979,17354707,"Cotton-Flickstoff iron-on, 11 x 25cm, beige","Cotton-iron-on patch, 11 x 25cm, beige",Outer carton: 10 pieces,,HDZ-07052,Heinrich Duscha OHG,H. Duscha oHG,"Ribbons, Cord, Patches,Ribbons, Cord, Patches",EU,00690856,00690856HDZ-07052


Join fields title and description

In [13]:
descriptions_en['text_all'] = descriptions_en.bezeichnung + ' ' + descriptions_en.beschreibung

In [14]:
descriptions_en.sample(25)

,artikelID,bezeichnung,beschreibung,artikelLangText,markenstichwort,artikelnummer,faname1,ausRufname,shopGroups,marktplatz,firmaID,anbieter_artikelnummer,text_all
853775,12603554,"dc you and me taupe, 200x200 / 220","<strong>DC YOU AND ME TAUPE, Size: 200x200 / 2...",Colors may deviate from the images some time,NIGHTLIFE,900865,Norien Safdar vof/ ho Ambianzz Bedding,Ambianzz,"microfiber,duvet covers",EU,00659483,00659483900865,"dc you and me taupe, 200x200 / 220 <strong>DC ..."
505690,8349721,"Snood Viscose, snood, Loop, Schal",Snood 90x180cm,Elegant snood made from the finest fabric ensu...,Lily Collection,lc18222.1,Luda GmbH,Lily Collection,"Scarves,Snoods Viscose,Fashion &amp; Accessori...",EU,00570741,00570741lc18222.1,"Snood Viscose, snood, Loop, Schal Snood 90x180cm"
277186,4214562,Eggs Forming,"Molding plastic eggs, which excites do the dis...",<html> There are two practical reasons this pl...,Tojásformázó,TOJ,Bradstock SRO,Bradstock SRO,,EU,00441523,00441523TOJ,"Eggs Forming Molding plastic eggs, which excit..."
1690755,19647486,canvas parchment leaves 51x79cm,canvas parchment leaves 51x79cm <br><br> Dimen...,,,30625,ArteRegal by zentrada.distribution,Arte Regal,"Discontinued,Discontinued",EU,00808353,0080835330625,canvas parchment leaves 51x79cm canvas parchme...
161096,2866238,SPRING DIADEMA PITO,SPRING DIADEMA PITO,,"EROTICO, FIESTA, JUGUETE, SEXO, REGALO",1018,"Diablo Picante, S.L.",Diablo Picante,"Ornaments, Lubricants, Toys,Couple Accessories",EU,00252361,002523611018,SPRING DIADEMA PITO SPRING DIADEMA PITO
850247,12338414,Hybrid Gel Nail Enamel 11ml cherry nr06,Delia Cosmetics Coral Hybrid Gel Nail Enamel N...,,Delia,5901350455562,A & T Serwis Tomasz Koc,TJK Cosmetics,"nail polish,Manicure &amp; Pedicure",EU,00658915,006589155901350455562,Hybrid Gel Nail Enamel 11ml cherry nr06 Delia ...
669279,10751641,PacMan Ghost Lamp Usb,PacMan Ghost Lamp Usb,<p> This lamp Usb PacMan Ghost offers two mode...,,1832,SARL KAS DESIGN,KAS DESIGN,,EU,00503457,005034571832,PacMan Ghost Lamp Usb PacMan Ghost Lamp Usb
1016150,14015790,Présentoire empty balmain pen + 12 clip,<strong>Présentoire empty balmain pen + 12 cli...,,PIERRE BALMAIN,40982180,PolyFlame by zentrada.distribution,Polyflame,"empty Displays,PIERRE BALMAIN,Lighters,Brands",EU,00698787,0069878740982180,Présentoire empty balmain pen + 12 clip <stron...
1106863,14893787,Malpka pendant in a wagon,Monkey monkey pendant <br/><br /> Joyful party...,,,REK-DFP84,Aptex Jóźwiak Spółka Jawna,AMJ,"To the baby,Toys",EU,00564429,00564429REK-DFP84,Malpka pendant in a wagon Monkey monkey pendan...
1017883,14028749,"box greedy ice, 1-times assorted","box greedy ice, 1-times assorted <br><br> - Pr...",,LILY COOK,KDO8538,cmp by zentrada.distribution,CMP Paris,"Utensils,Cooked",EU,00694944,00694944KDO8538,"box greedy ice, 1-times assorted box greedy ic..."


In [15]:
descriptions_en = descriptions_en[['anbieter_artikelnummer', 'text_all']]

In [16]:
descriptions_en.sample(10)

,anbieter_artikelnummer,text_all
1625812,0072974170925V2,square card Birthday I Know I&#39;m Cute <stro...
1058626,00561756Schal-QX1701-184WJ grau,Scarf uni fringes III gray Scarf uni fringes III
1045580,00183826VARKEN5053164003610,Plush Pig 30cm Plush Pig 30cm. <br /><br /> Si...
1455659,000040058100072-200,"Stainless steel mesh tape, thickness 1.5 mm, s..."
1195734,0036444294-2593,"Wooden Picture Frame, Simple Style, 13 x 18 cm..."
1070331,00481425I385704,Small stylus for a capacitive screen. Small st...
340749,000790584105933,"ELC dimmable energy saving lamp, E14, 230V, 11..."
846191,00382446QEL10024,"Tableware, plastic sets of Disney Minnie Table..."
578548,002274800232113407,Pendant Item No. 0232113407 Pendant - Turquois...
1016150,0069878740982180,Présentoire empty balmain pen + 12 clip <stron...


Clean data - extract from html

In [17]:
beschreibung = []
for h in descriptions_en['text_all']:
    try:
        b = BeautifulSoup(h, features = "html.parser")
        beschreibung.append(b.get_text())
    except:
        beschreibung.append(None)

descriptions_en.loc[:,'text_clean'] = beschreibung

In [18]:
descriptions_en.drop("text_all", axis = 1, inplace = True)

In [19]:
items_text = pd.merge(items_df, descriptions_en, how = "left", on = "anbieter_artikelnummer")

In [20]:
len(items_text)

749032

Count the average number of words for each description - is not cleaned yet, still includes numbers, short words, and stopwords

In [23]:
count = items_text['text_all'].str.split().str.len()

In [24]:
count.describe()

count    698190.000000
mean         39.818981
std          32.329856
min           2.000000
25%          18.000000
50%          30.000000
75%          50.000000
max         232.000000
Name: text_all, dtype: float64

### Embedd words and create average vec

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Create stopwords

In [24]:
my_stop_words = STOPWORDS.union(set(['size', 'color', 'material', 'product', 'dimension', 'length', 'package', 'brand', 
                     'pack', 'width', 'piece', 'height', 'quality', 'group', 'high', 'model', 'article', 'assort',
                     'price', 'weight', 'colour', 'products', 'type', 'design', 'diameter']))

Preprocess the words:
- remove stopwords
- remove words less than 4 characters
- lemmatize words

In [25]:
def preprocess(raw):
    text = BeautifulSoup(raw, features = "html").get_text() # extract from html
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in my_stop_words and len(token) > 3: # standard stopwords and remove shorter 3 
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return result

In [27]:
# test
preprocess(items_text.text_clean.tolist()[2])

['erasers',
 'rainbow',
 'loom',
 'band',
 'mega',
 'erasers',
 'rainbow',
 'loom',
 'band',
 'mega',
 'silicone',
 'rubber',
 'band',
 'crochet',
 'hook',
 'clip',
 'loom',
 'mini',
 'loom',
 'colorful',
 'rubber',
 'band',
 'piece',
 'instructions',
 'demonstration',
 'state']

Embedd the words and calculate average vector

In [28]:
model = Word2Vec.load('data/models/word2vec.model')
word_vectors = model.wv # could also just save this later to speed up and decrease memory ! 

def get_mean_vector(w2v_model, word_vecs, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word_vecs.vocab] 
    if len(words) >= 1:
        return np.mean(w2v_model[words], axis=0) # average the vecs
    else:
        return []

Apply both functions to data 

In [37]:
items_text.reset_index(inplace = True, drop = True)

In [39]:
text_vec_list = []

for n in range(len(items_text)):
        try:
            prepro = preprocess(items_text.text_clean[n])
            ls = get_mean_vector(model, word_vectors, prepro).tolist()
            text_vec_list.append(ls)
        except: 
            ls = [0] * 150
            text_vec_list.append(ls)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [41]:
text_vec_list[0]

[-1.075660228729248,
 -1.6229168176651,
 0.15882129967212677,
 0.382991224527359,
 -0.6077048778533936,
 -0.11547023057937622,
 -0.36997073888778687,
 -0.08155835419893265,
 -0.6791936755180359,
 -0.010646536946296692,
 0.5592737197875977,
 -0.47856977581977844,
 -0.6405319571495056,
 0.5362224578857422,
 -0.4863002300262451,
 -0.6106874346733093,
 1.1538559198379517,
 -0.05181221663951874,
 -0.23332789540290833,
 0.03226054832339287,
 -0.18601317703723907,
 -0.5949712991714478,
 -0.17790579795837402,
 0.19484683871269226,
 0.429412841796875,
 -0.640110969543457,
 0.0130864717066288,
 -0.030461039394140244,
 0.6104621887207031,
 0.27940094470977783,
 0.38617175817489624,
 0.007335186004638672,
 -0.2745586633682251,
 0.041437193751335144,
 0.16562867164611816,
 0.9861876368522644,
 0.6554923057556152,
 0.41171133518218994,
 0.336512953042984,
 0.5873960852622986,
 -0.8517641425132751,
 -0.824733316898346,
 0.2890174984931946,
 0.4773341417312622,
 -0.9732609987258911,
 0.048604778945446

In [42]:
vecs = pd.DataFrame({"text_vec": text_vec_list})

In [ ]:
item_text_vec = pd.merge(items_text, vecs, left_index = True, right_index= True)

In [ ]:
item_text_vec.to_pickle('data/processed_data/articles_info.pkl')